In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jieba.analyse
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import jieba.posseg as pseg
from tqdm import tqdm
import collections

In [2]:
dfc = pd.read_csv('..\data\data_C.csv',index_col = 0,dtype = {'simple review':str,'review':str,})

In [3]:
dfc

,car type,purch time,purch place,purch price,avg rate,publish time,score,purch purp,simple review,review,cum mileage
0,2019款奔驰C级C 300 L 运动版,2019-01-01,上海星瀚汽车维修服务有限公司（奔驰） [上海市],35万 (厂商指导价 47.48万),15L/100km,2019-03-07,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],外观惊艳，内饰完美，30万级别绝对无敌手，音响比较hao啊，后备箱有点小了，我自己的话是够用...,[最满意]\n绚丽的立标外观，迷离的多条式轮圈，车身流畅的线条，光看前脸，回头率比较高，从侧...,NaN
1,2019款奔驰C级C 300 L 运动版,2019-01-01,烟台之星汽车服务有限公司 [山东省],49万 (厂商指导价 47.48万),9L/100km,2019-03-07,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],绚丽的立标外观，迷离的多条式轮圈，车身流畅的线条，光看前脸，回头率比较高，从侧面看就有种蓄势...,[最满意]\n外观惊艳，内饰完美，超乎我的预期\n[不满意]\n后备箱有点小了，我自己的话是...,NaN
2,2018款奔驰C级C 200 L,2018-11-01,北京之星汽车服务有限公司 [北京市],30万 (厂商指导价 35.08万),9L/100km,2019-02-14,"{'外观': '5', '内饰': '4', '空间': '5', '舒适': '5', '...",['购物'],不管时外型还是配置和是操纵感，都比较满足我的要求，我想要的它都给了我，所以我选择,[最满意]\n最喜欢的还是奔驰的大标，就是那么的大气，透着一股高端范。说白了就是儿时梦想。车...,NaN
3,2018款奔驰C级C 200 L 运动型 成就特别版,2018-07-01,南京江北之星汽车服务有限公司 [江苏省],32.38万 (厂商指导价 35.38万),10L/100km,2018-12-10,"{'外观': '5', '内饰': '5', '空间': '4', '舒适': '4', '...","['上班', '购物']",这车可没有性价比，比起宝马奥迪真心贵，没什么优惠，奔驰卖的还硬，但是保值呀,[最满意]\n最满意的是外观内饰，再一个就是最新的9速自动变速箱，很聪明，唯一有一个理由就是...,NaN
4,2018款奔驰C级C 200 L 运动版,2018-01-01,利星行(北京)汽车有限公司 [北京市],35.8万 (厂商指导价 35.08万),10.6L/100km,2018-11-25,"{'外观': '5', '内饰': '4', '空间': '4', '舒适': '5', '...","['上班', '购物', '自驾游']",外观简单大气，自我感觉内饰好看有设计感，加长版后排空间大，后备箱空间也比较满意，平均油耗比较...,[最满意]\n用车时间小一年，这款车型很多人都有所了解，现在发一下用车以来的自我感觉以及评价...,NaN
5,2008款奔驰C级C 200K 时尚型,2008-05-01,[河北省],41.98万 (厂商指导价 万),8.8L/100km,2018-11-21,"{'外观': '5', '内饰': '4', '空间': '3', '舒适': '4', '...","['上班', '购物', '接送小孩', '自驾游', '跑长途']",很喜欢这个品牌，一生值得拥有。作为一个男人，我感觉在条件许可的情况下必须拥有。首先是品牌实力...,[最满意]\n外形。射手座男，本身就喜欢浪漫不羁，自由奔放，热爱生活。崇尚美追求美。遗传基因...,NaN
6,2019款奔驰C级C 180 L 时尚型运动版,2018-10-01,[北京市],26万 (厂商指导价 31.58万),10L/100km,2018-11-20,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '购物', '接送小孩', '自驾游', '跑长途']",最满意的就是外观和内饰了，在bba里真的就算很漂亮的了。Mercedesme非常不错，可以远...,[最满意]\n最满意的就是外观和内饰了，在bba里真的就算很漂亮的了。Mercedesme非...,NaN
7,2017款奔驰C级C 200 运动版,2017-11-19,重庆市合翘汽车销售服务有限公司 [重庆市],38.38万 (厂商指导价 31.48万),8.7L/100km,2017-11-29,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],"不错,美中不足要加价,选配较难.具体我写了一篇作业.http://www.xcar.com....",[最满意]\n外观.奔驰的内饰一直口碑不错，柏林之声和真皮内饰更是锦上添花。柏林之声的高低音...,NaN
8,2015款奔驰C级C 300 L,2016-05-25,x上海闵星汽车服务有限公司金桥分公司 [上海市],38.5万 (厂商指导价 48.90万),8.3L/100km,2016-09-06,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '购物', '接送小孩', '自驾游', '跑长途']",NaN,[最满意]\n外观漂亮，时尚，内饰科技感强，比较豪华，动力比较足，操控精准。奔驰的整体质量比...,NaN
9,2010款奔驰C级C 180K 经典版,2011-12-06,苏州利星汽车服务有限公司 [江苏省],30万 (厂商指导价 30.80万),11L/100km,2016-08-03,"{'外观': '5', '内饰': '5', '空间': '1', '舒适': '3', '...",['上班'],NaN,[最满意]\n品牌，没什么可说的，就是冲着这品牌去的啦。其次是底盘，过完什么的信心十足。\n...,NaN


In [4]:
sum(pd.notnull(dfc['review'])) # 3062 lines with review
dfc_review = dfc[pd.notnull(dfc['review'])] 
dfc_review = dfc_review.reset_index(drop = True)
dfc_review

3062

,car type,purch time,purch place,purch price,avg rate,publish time,score,purch purp,simple review,review,cum mileage
0,2019款奔驰C级C 300 L 运动版,2019-01-01,上海星瀚汽车维修服务有限公司（奔驰） [上海市],35万 (厂商指导价 47.48万),15L/100km,2019-03-07,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],外观惊艳，内饰完美，30万级别绝对无敌手，音响比较hao啊，后备箱有点小了，我自己的话是够用...,[最满意]\n绚丽的立标外观，迷离的多条式轮圈，车身流畅的线条，光看前脸，回头率比较高，从侧...,NaN
1,2019款奔驰C级C 300 L 运动版,2019-01-01,烟台之星汽车服务有限公司 [山东省],49万 (厂商指导价 47.48万),9L/100km,2019-03-07,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],绚丽的立标外观，迷离的多条式轮圈，车身流畅的线条，光看前脸，回头率比较高，从侧面看就有种蓄势...,[最满意]\n外观惊艳，内饰完美，超乎我的预期\n[不满意]\n后备箱有点小了，我自己的话是...,NaN
2,2018款奔驰C级C 200 L,2018-11-01,北京之星汽车服务有限公司 [北京市],30万 (厂商指导价 35.08万),9L/100km,2019-02-14,"{'外观': '5', '内饰': '4', '空间': '5', '舒适': '5', '...",['购物'],不管时外型还是配置和是操纵感，都比较满足我的要求，我想要的它都给了我，所以我选择,[最满意]\n最喜欢的还是奔驰的大标，就是那么的大气，透着一股高端范。说白了就是儿时梦想。车...,NaN
3,2018款奔驰C级C 200 L 运动型 成就特别版,2018-07-01,南京江北之星汽车服务有限公司 [江苏省],32.38万 (厂商指导价 35.38万),10L/100km,2018-12-10,"{'外观': '5', '内饰': '5', '空间': '4', '舒适': '4', '...","['上班', '购物']",这车可没有性价比，比起宝马奥迪真心贵，没什么优惠，奔驰卖的还硬，但是保值呀,[最满意]\n最满意的是外观内饰，再一个就是最新的9速自动变速箱，很聪明，唯一有一个理由就是...,NaN
4,2018款奔驰C级C 200 L 运动版,2018-01-01,利星行(北京)汽车有限公司 [北京市],35.8万 (厂商指导价 35.08万),10.6L/100km,2018-11-25,"{'外观': '5', '内饰': '4', '空间': '4', '舒适': '5', '...","['上班', '购物', '自驾游']",外观简单大气，自我感觉内饰好看有设计感，加长版后排空间大，后备箱空间也比较满意，平均油耗比较...,[最满意]\n用车时间小一年，这款车型很多人都有所了解，现在发一下用车以来的自我感觉以及评价...,NaN
5,2008款奔驰C级C 200K 时尚型,2008-05-01,[河北省],41.98万 (厂商指导价 万),8.8L/100km,2018-11-21,"{'外观': '5', '内饰': '4', '空间': '3', '舒适': '4', '...","['上班', '购物', '接送小孩', '自驾游', '跑长途']",很喜欢这个品牌，一生值得拥有。作为一个男人，我感觉在条件许可的情况下必须拥有。首先是品牌实力...,[最满意]\n外形。射手座男，本身就喜欢浪漫不羁，自由奔放，热爱生活。崇尚美追求美。遗传基因...,NaN
6,2019款奔驰C级C 180 L 时尚型运动版,2018-10-01,[北京市],26万 (厂商指导价 31.58万),10L/100km,2018-11-20,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '购物', '接送小孩', '自驾游', '跑长途']",最满意的就是外观和内饰了，在bba里真的就算很漂亮的了。Mercedesme非常不错，可以远...,[最满意]\n最满意的就是外观和内饰了，在bba里真的就算很漂亮的了。Mercedesme非...,NaN
7,2017款奔驰C级C 200 运动版,2017-11-19,重庆市合翘汽车销售服务有限公司 [重庆市],38.38万 (厂商指导价 31.48万),8.7L/100km,2017-11-29,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...",['上班'],"不错,美中不足要加价,选配较难.具体我写了一篇作业.http://www.xcar.com....",[最满意]\n外观.奔驰的内饰一直口碑不错，柏林之声和真皮内饰更是锦上添花。柏林之声的高低音...,NaN
8,2015款奔驰C级C 300 L,2016-05-25,x上海闵星汽车服务有限公司金桥分公司 [上海市],38.5万 (厂商指导价 48.90万),8.3L/100km,2016-09-06,"{'外观': '5', '内饰': '5', '空间': '5', '舒适': '5', '...","['上班', '购物', '接送小孩', '自驾游', '跑长途']",NaN,[最满意]\n外观漂亮，时尚，内饰科技感强，比较豪华，动力比较足，操控精准。奔驰的整体质量比...,NaN
9,2010款奔驰C级C 180K 经典版,2011-12-06,苏州利星汽车服务有限公司 [江苏省],30万 (厂商指导价 30.80万),11L/100km,2016-08-03,"{'外观': '5', '内饰': '5', '空间': '1', '舒适': '3', '...",['上班'],NaN,[最满意]\n品牌，没什么可说的，就是冲着这品牌去的啦。其次是底盘，过完什么的信心十足。\n...,NaN


In [5]:
def loadDataset(line):
    corpus = ''
    #加载停用词表
    stop = [line.strip().decode('utf-8') for line in open('..\data\stop_words_4.txt','rb').readlines() ]
    #无外观内饰空间舒适油耗动力操控配置性价比
    segs = jieba.lcut(line.strip(), cut_all=True)   #, cut_all=True
    segs = pseg.cut(line)
    final = ''
    for seg, flag in segs:
        #去停用词
        if seg not in stop:
            #去数词和去字符串
            if flag !='m' and flag !='x':
                    #输出分词
                final +=' '+ seg 
#             corpus.append(final)
    corpus += ' ' +final 
            
    return corpus
# dataset = loadDataset()

In [6]:
def tfidf(dataset):
    tv=TfidfVectorizer(min_df=0.,max_df=1.,norm='l2',use_idf=True,smooth_idf=True)
    tv_matrix=tv.fit_transform([dataset])
    #print(tv_matrix)
    tv_matrix=tv_matrix.toarray()
    #print(tv_matrix)
    vocab=tv.get_feature_names()
    df_com = pd.DataFrame(np.round(tv_matrix,2),columns=vocab).T.sort_values(by=[0],ascending=False)
    df_com.columns = ['tf-idf']
    return df_com[0:55]

In [7]:
total = ''
for i in dfc_review.review.unique():
    total += i
dataset = loadDataset(total)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\panji\AppData\Local\Temp\jieba.cache
Loading model cost 1.056 seconds.
Prefix dict has been built successfully.


In [8]:
tfidf(dataset)

,tf-idf
空间,0.37
内饰,0.37
油耗,0.33
外观,0.30
动力,0.30
操控,0.23
舒适,0.22
后排,0.19
配置,0.17
座椅,0.15


In [16]:
tfidf(dataset).index
top = ['空间', '内饰', '油耗', '外观', '动力', '操控', '舒适', '后排', '配置', '座椅', '高速', '性价比',
       '舒服', '方向盘', '驾驶', '设计', '发动机', '舒适性', '市区', '运动', '做工', '加速', '品牌',
       '前排', '转向', '油门', '异响', '中控', '起步', '超车', '豪华', '声音', '后备箱', '刹车', '自动', 
       '顿挫', '价格', '优惠', '车身', '漂亮', '车型', '底盘', '优雅',  '乘坐', '安全', '大灯', '天窗']

Index(['空间', '内饰', '油耗', '外观', '动力', '操控', '舒适', '后排', '配置', '座椅', '高速', '性价比',
       '舒服', '方向盘', '驾驶', '设计', '发动机', '舒适性', '市区', '运动', '做工', '加速', '品牌',
       '前排', '方面', '转向', '油门', '异响', '中控', '起步', '超车', '豪华', '声音', '后备箱', '个油',
       '确实', '刹车', '自动', '顿挫', '价格', '优惠', '车身', '漂亮', '车型', '那种', '东西', '以后',
       '底盘', '优雅', '相当', '大家', '乘坐', '安全', '大灯', '天窗'],
      dtype='object')

In [19]:
# top = tfidf(dataset).index

total_string = []
for i in dfc_review.review.tolist():
    have_string = {}
    for j in top:
        if j in i:
            have_string[i.index(j)] = j
    string = []
    for k in sorted(have_string.keys()):
        string.append(have_string[k])
    total_string.append(string)

In [20]:
df_tfidf = pd.DataFrame(columns=['words'])
df_tfidf['words'] = total_string
df_tfidf

,words
0,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
1,"[外观, 内饰, 后备箱, 设计, 中控, 空间, 后排, 舒适, 舒适性, 舒服, 油耗,..."
2,"[配置, 品牌, 外观, 设计, 车身, 驾驶, 内饰, 中控, 空间, 前排, 后排, 乘..."
3,"[外观, 内饰, 自动, 漂亮, 天窗, 后排, 起步, 顿挫, 运动, 豪华, 空间, 操..."
4,"[车型, 外观, 油耗, 中控, 异响, 驾驶, 声音, 天窗, 内饰, 车身, 设计, 空..."
5,"[外观, 大灯, 内饰, 座椅, 空间, 前排, 后备箱, 漂亮, 舒适, 舒服, 底盘, ..."
6,"[外观, 内饰, 漂亮, 优惠, 异响, 后排, 座椅, 舒服, 空间, 舒适, 底盘, 自..."
7,"[外观, 内饰, 方向盘, 舒服, 大灯, 设计, 安全, 运动, 座椅, 配置, 声音, ..."
8,"[外观, 漂亮, 内饰, 豪华, 动力, 操控, 顿挫, 方向盘, 转向, 异响, 配置, ..."
9,"[品牌, 底盘, 空间, 外观, 优雅, 内饰, 前排, 后排, 舒适, 舒服, 乘坐, 油..."


In [21]:
keywords_array=[]
for index, row in df_tfidf.iterrows():
    keywords=row['words']
    for kw in keywords:
        keywords_array.append((kw.strip(' '), row['words']))
kw_df = pd.DataFrame(keywords_array).rename(columns={0:'keyword', 1:'keywords'})
kw_df

,keyword,keywords
0,外观,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
1,车身,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
2,起步,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
3,油门,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
4,内饰,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
5,中控,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
6,设计,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
7,空间,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
8,后排,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."
9,舒适,"[外观, 车身, 起步, 油门, 内饰, 中控, 设计, 空间, 后排, 舒适, 舒适性, ..."


In [22]:
document = kw_df.keywords.tolist()
names = kw_df.keyword.tolist()

document_array = []
for item in tqdm(document):
    document_array.append((item))

occurrences = collections.OrderedDict((name, collections.OrderedDict((name, 0) for name in names)) for name in tqdm(names))

# Find the co-occurrences:
for l in tqdm(document_array):
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1

co_occur = pd.DataFrame.from_dict(occurrences)

100%|██████████████████████████████████████████████████████████████████████████| 55629/55629 [00:10<00:00, 5201.02it/s]


In [23]:
co_occur.to_csv('..\data\co-occurancy_matrix_C.csv')

In [24]:
co_occur

,外观,车身,起步,油门,内饰,中控,设计,空间,后排,舒适,...,异响,声音,安全,刹车,大灯,底盘,发动机,做工,市区,优雅
中控,18695,5687,5473,6066,18805,0,8281,18476,15782,16517,...,5561,3418,4292,5163,3779,3497,6306,6767,6417,3357
乘坐,12156,4468,4205,4457,12217,5603,6081,12236,10731,10424,...,2765,2090,3244,3698,2325,2979,4500,5590,4577,2026
价格,15304,4334,4369,4644,15359,5563,6134,15207,11885,12830,...,3608,2816,3189,2901,3265,2805,5946,6011,6580,2607
优惠,11734,2359,3035,4504,11771,3066,3639,11696,8985,10698,...,2052,2231,2034,2671,2371,2220,3924,5603,5078,1903
优雅,8601,3202,2962,3097,8669,3357,3983,8646,7012,7134,...,2226,1344,2396,2573,2612,2271,3366,3554,4045,0
做工,20476,5655,5078,8451,20846,6767,8274,20545,15979,17199,...,4601,3119,5755,5667,4367,4617,7520,0,8997,3554
内饰,53726,12461,14267,18052,0,18805,18772,53602,41668,47499,...,12769,9793,11157,13065,10633,10002,18103,20846,21529,8669
刹车,12892,3037,4089,5636,13065,5163,4923,12929,9767,11061,...,4742,3128,4410,0,3382,3153,5203,5667,5570,2573
前排,19285,5627,5984,6328,19411,7669,8207,19493,18733,16298,...,4606,3422,4863,4505,4148,4370,6360,8417,7957,3324
加速,17156,4314,5707,8079,17285,5470,6375,17090,13489,14633,...,3925,4215,4683,5011,4108,3881,7139,7849,6837,3194
